In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# Import Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd
from tensorflow.keras.layers import Input, Embedding,Dense,  LSTM

In [ ]:
# !pip install datasets==1.18.1
from datasets import load_dataset
dataset = load_dataset("cfilt/iitb-english-hindi")

## Preparing Data

### TASK: Create Encder-Decoder LSTM model to convert Hindi sentences to English sentences.  

### 2. Read Data : IIT Bombay 

In [ ]:
def createDataset(dataset, data_size):
    pairs=[] 
    for translation_pair in dataset["train"]["translation"]:
        source_sentence = translation_pair["hi"]
        target_sentence = translation_pair["en"]
        pairs.append([source_sentence, target_sentence])
    lines= pd.DataFrame(columns=[ "hindi","eng"], data=pairs)
    lines= lines[:data_size]
    return lines
      


### 3) Preparing input data for the Encoder ( `encoder_input_data` )

In [ ]:
def createDatasetForEncoder(lines):
    hindi_lines = list()
    for line in lines.hindi:
        hindi_lines.append( line ) 
        
    tokenizer = preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts( hindi_lines ) 
    tokenized_hindi_lines = tokenizer.texts_to_sequences( hindi_lines ) 
    length_list = list()

    for token_seq in tokenized_hindi_lines:
        length_list.append( len( token_seq ))

    max_input_length = np.array( length_list ).max()
    padded_hindi_lines = preprocessing.sequence.pad_sequences( tokenized_hindi_lines , maxlen=max_input_length , padding='post' )
    encoder_input_data = np.array( padded_hindi_lines )
   
    hindi_word_dict = tokenizer.word_index
    num_hindi_tokens = len( hindi_word_dict )+1
    # print("Hindi Dictionary" ,hindi_word_dict  )
    return max_input_length, num_hindi_tokens, encoder_input_data, hindi_word_dict
   


                

### Create Dataset for Decoder

### 4) Preparing input data for the Decoder ( `decoder_input_data` )

In [ ]:
def createDatasetForDecoder(lines):
    eng_lines = list()
    for line in lines.eng:
        eng_lines.append( '<START> ' + line + ' <END>' )  

    tokenizer = preprocessing.text.Tokenizer(oov_token=1)
    tokenizer.fit_on_texts( eng_lines ) 
    tokenized_eng_lines = tokenizer.texts_to_sequences( eng_lines ) 

    length_list = list()
    for token_seq in tokenized_eng_lines:
        length_list.append( len( token_seq ))
    max_output_length = np.array( length_list ).max()
    print( 'English max length is {}'.format( max_output_length ))

    padded_eng_lines = preprocessing.sequence.pad_sequences( tokenized_eng_lines , maxlen=max_output_length, padding='post' )
    decoder_input_data = np.array( padded_eng_lines  )
    print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

    eng_word_dict = tokenizer.word_index
    num_eng_tokens = len( eng_word_dict )+1
    print( 'Number of English tokens = {}'.format( num_eng_tokens))

    return max_output_length, num_eng_tokens, decoder_input_data, eng_word_dict, tokenized_eng_lines

### 5) Preparing target data for the Decoder ( decoder_target_data ) 

In [ ]:
def createDecoderTargetData(tokenized_eng_lines, max_output_length, num_eng_tokens):
    decoder_target_data = list()
    for token_seq in tokenized_eng_lines:
        decoder_target_data.append( token_seq[ 1 : ] ) 
        
    padded_eng_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
    onehot_eng_lines = utils.to_categorical( padded_eng_lines , num_eng_tokens )
    decoder_target_data = np.array( onehot_eng_lines )
    print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))
    return decoder_target_data

### 1) Defining the Encoder-Decoder model

In [ ]:
def createEncoderDecoderModel(max_input_length, max_output_length, num_hindi_tokens, num_eng_tokens):
    encoder_inputs = Input(shape=( max_input_length ,  ))
    encoder_embedding = Embedding( num_hindi_tokens, 256 , mask_zero=True ) (encoder_inputs)
    encoder_outputs , state_h , state_c = LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
    encoder_states = [ state_h , state_c ]

    decoder_inputs = Input(shape=( max_output_length , ))
    decoder_embedding = Embedding( num_eng_tokens, 256 , mask_zero=True) (decoder_inputs)
    decoder_lstm = LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
    decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
    decoder_dense = Dense( num_eng_tokens , activation=tf.keras.activations.softmax ) 
    output = decoder_dense ( decoder_outputs )

    model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')
    return model



In [ ]:
data_size=100000
pool_oftexts=createDataset(dataset=dataset,data_size=data_size)

In [ ]:
model_size=[1000, 2000,3000,4000,5000]
my_list_of_trained_models=[] 

In [ ]:
for item in model_size:
    lines= pool_oftexts[:item]
    lines.count()
    max_input_length, num_hindi_tokens, encoder_input_data, input_dict= createDatasetForEncoder(lines=lines)
    max_output_length, num_eng_tokens, decoder_input_data, eng_word_dict, tokenized_eng_lines =createDatasetForDecoder(lines=lines)
    decoder_target_data= createDecoderTargetData(tokenized_eng_lines, max_output_length, num_eng_tokens)
    model= createEncoderDecoderModel(max_input_length, max_output_length, num_hindi_tokens, num_eng_tokens)
    mc = tf.keras.callbacks.ModelCheckpoint('my_nmt_model_min_loss.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    history = model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=250, epochs=10 ,validation_split = 0.1, callbacks=[mc], verbose=1    ) 
    my_list_of_trained_models.append(model)
    model.save( str(item)+'model.h5' ) 


## Think what should be done with variable history
      

In [ ]:
my_list_of_trained_models[0]

### 2) Training the model
We train the model for a number of epochs with RMSprop optimizer and categorical crossentropy loss function.

In [ ]:
from tensorflow import keras
saved_model = keras.models.load_model('1000model.h5')
saved_model.summary()
# saved_model = keras.models.load_model('5000model.h5')
# saved_model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(saved_model, to_file='modelsummary.png', show_shapes=True, show_layer_names=True)

In [ ]:
encoder_inputs = saved_model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = saved_model.layers[4].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)
latent_dim = 256  # Note: may be need to save in drive as well


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])

## Inferencing on the models

### 1) Defining inference models
We create inference models which help in predicting translations.

**Encoder inference model** : Takes the English sentence as input and outputs LSTM states ( `h` and `c` ).

**Decoder inference model** : Takes in 2 inputs, one are the LSTM states ( Output of encoder model ), second are the French input seqeunces ( ones not having the `<start>` tag ). It will output the translations of the English sentence which we fed to the encoder model and its state values.





In [ ]:

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 256 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


### 2) Making some translations


1.   First, we take a English sequence and predict the state values using `enc_model`.
2.   We set the state values in the decoder's LSTM.
3.   Then, we generate a sequence which contains the `<start>` element.
4.   We input this sequence in the `dec_model`.
5.   We replace the `<start>` element with the element which was predicted by the `dec_model` and update the state values.
6.   We carry out the above steps iteratively till we hit the `<end>` tag or the maximum sequence length.







In [ ]:

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
          print("word ", word,hindi_word_dict.get(word,1) )
          my_word=  hindi_word_dict.get(word,1)
          tokens_list.append(my_word) 
  
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')


In [ ]:

enc_model , dec_model = make_inference_models()

#encoder_input_data.shape[0] 
for epoch in range(1 ):
    states_values = enc_model.predict( str_to_tokens("मुझे खाने से प्यार है" ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = eng_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in eng_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    # print("Decoded Traslation ", decoded_translation )
    print(f"{bcolors.OKGREEN}Decoded Traslation: { decoded_translation}{bcolors.ENDC}")

    

### Save Encoder and Decoder Parameters

In [ ]:
%cd "drive/MyDrive/Machine Learning/GitHub Projects/machine-translation/models/"
model_name = str(lines.count().eng)
!mkdir $model_name
!ls
# save encoder model
enc_model.save( model_name+'/enc_model.h5' ) 
# save decoder model
dec_model.save( model_name+'/dec_model.h5' ) 
# save  model
saved_model.save( model_name+'/model.h5' ) 

In [ ]:
# encoder parameters
encoder_parameters={
    'max_encoder_seq_length': max_input_length,
    'num_encoder_tokens': num_hindi_tokens,
    
   
}
encoder_dictionary=  hindi_word_dict


# decoder parameters
decoder_parameters={
    'max_decoder_seq_length':  max_output_length,
    'num_decoder_tokens': num_eng_tokens,

}

decoder_dictionary=  eng_word_dict



In [ ]:
parameters=model_name+"/parameters/"
dictionaries=model_name+"/dictionaries/"
!mkdir $parameters
!mkdir $dictionaries


In [ ]:
import pickle
paramters_path=parameters
dictionaries_path=dictionaries

# save encoder parameter
with open(paramters_path+'encoder_parameters.pickle', 'wb') as handle:
    pickle.dump(encoder_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

# save encoder dictionary
with open(dictionaries_path+'encoder_dictionary.pickle', 'wb') as handle:
    pickle.dump(encoder_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

# save encoder parameter
with open(paramters_path+'decoder_parameters.pickle', 'wb') as handle:
    pickle.dump(decoder_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

# save encoder parameter
with open(dictionaries_path+'decoder_dictionary.pickle', 'wb') as handle:
    pickle.dump(decoder_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)    

